In [13]:
import robomimic.utils.tensor_utils as TensorUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.train_utils as TrainUtils
from bc_benchmark_algos.dataset.robomimic import RobomimicDataset
from bc_benchmark_algos.rollout_env.robomimic import RobomimicRolloutEnv
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
import torch
import numpy as np
import json

In [14]:
# test tensor utils
x = {"obs": {"agentview_image": np.random.randn(3, 84, 84)}}
x = TensorUtils.to_tensor(x)
assert isinstance(x["obs"]["agentview_image"], torch.Tensor) 
x = TensorUtils.to_batch(x)
assert x["obs"]["agentview_image"].shape == (1, 3, 84, 84)
x = TensorUtils.to_sequence(x)
assert x["obs"]["agentview_image"].shape == (1, 1, 3, 84, 84)
x = TensorUtils.repeat_seq(x=x, k=10)
assert x["obs"]["agentview_image"].shape == (1, 10, 3, 84, 84)
x = TensorUtils.slice(x=x, dim=1, start=0, end=5)
assert x["obs"]["agentview_image"].shape == (1, 5, 3, 84, 84)
y = TensorUtils.shift_seq(x=x, k=1)
assert torch.equal(x["obs"]["agentview_image"][:, 0, :], y["obs"]["agentview_image"][:, 1, :])
y = TensorUtils.shift_seq(x=x, k=-1)
assert torch.equal(x["obs"]["agentview_image"], y["obs"]["agentview_image"])

In [22]:
# setup config
config_path = "../config/bc_rnn.json"
dataset_path = "../../datasets/test/square_ph.hdf5"
output_dir = "output"
ext_cfg = json.load(open(config_path, 'r'))
config = config_factory(ext_cfg["algo_name"])
with config.unlocked():
    config.update(ext_cfg)
config.train.data = dataset_path
config.train.output_dir = output_dir
config.train.frame_stack = 5
config.train.seq_length = 1
config.lock()

ObsUtils.initialize_obs_utils_with_config(config)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos', 'robot0_eef_quat']
using obs modality: rgb with keys: ['agentview_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


In [23]:
# create validset
dataset_path = "../../datasets/test/square_ph.hdf5"
validset = RobomimicDataset.dataset_factory(
    config=config,
    obs_group_to_keys=ObsUtils.OBS_GROUP_TO_KEYS[config.algo_name], 
    filter_by_attribute="valid"
    )

RobomimicDataset: loading dataset into memory...
100%|██████████| 20/20 [00:00<00:00, 202.33it/s]
RobomimicDataset: caching get_item calls...
100%|██████████| 2989/2989 [00:00<00:00, 3587.91it/s]


In [36]:
# create rollout env
_, _, video_dir = TrainUtils.get_exp_dir(config)
rollout_env = RobomimicRolloutEnv(config=config, validset=validset, video_dir=video_dir)
print(rollout_env.env_meta)

REMOVING


INFO: Probing, EGL cannot run on this device


Created environment with name NutAssemblySquare
Action size is 7
{'env_name': 'NutAssemblySquare', 'env_version': '1.4.1', 'type': 1, 'env_kwargs': {'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_camera_obs': True, 'control_freq': 20, 'controller_configs': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'control_delta': True, 'interpolation': None, 'ramp_ratio': 0.2}, 'robots': ['Panda'], 'camera_depths': False, 'camera_heights': 84, 'camera_widths': 84, 'reward_shaping': False, 'camera_names': ['agentview'], 'render_gpu_device_id': 0}}


In [35]:
# test inputs_from_initial_obs
demo_id = validset.demos[0]
initial_state = dict(states=validset.hdf5_file[f"data/{demo_id}/states"][0])
initial_state["model"] = validset.hdf5_file[f"data/{demo_id}"].attrs["model_file"]
rollout_env.env.reset()
obs = rollout_env.env.reset_to(initial_state)
obs, _, _, _ = rollout_env.env.step(np.zeros(7))
print(obs["agentview_image"].shape)
inputs = rollout_env.inputs_from_initial_obs(obs=obs)
print(inputs["obs"]["agentview_image"].shape)
assert inputs["obs"]["agentview_image"].shape == (1, config.train.frame_stack+1, 3, 84, 84)


(5, 3, 84, 84)
torch.Size([1, 6, 5, 3, 84, 84])


AssertionError: 